In [2]:
! pip install geopandas
! pip install pandas
! pip install pygeos
! pip install boto
! pip install s3fs
! pip install shapely
! pip install gcsfs
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import gzip
import time
from itertools import groupby
import geopandas as gpd

/opt/conda/lib/python3.10/site-packages/geopandas/_compat.py:123: UserWarning: The Shapely GEOS version (3.11.1-CAPI-1.17.1) is incompatible with the GEOS version PyGEOS was compiled with (3.10.4-CAPI-1.16.2). Conversions between both will be slow.
  warnings.warn(
/tmp/ipykernel_88/1029240406.py:14: UserWarning: Shapely 2.0 is installed, but because PyGEOS is also installed, GeoPandas will still use PyGEOS by default for now. To force to use and test Shapely 2.0, you have to set the environment variable USE_PYGEOS=0. You can do this before starting the Python process, or in your code before importing geopandas:

import os
os.environ['USE_PYGEOS'] = '0'
import geopandas

In a future release, GeoPandas will switch to using Shapely by default. If you are using PyGEOS directly (calling PyGEOS functions on geometries from GeoPandas), this will then stop working and you are encouraged to migrate from PyGEOS to Shapely 2.0 (https://shapely.readthedocs.io/en/latest/migration_pygeos.html).
  i

In [3]:
! pip install ipynb
import ipynb

  Using cached ipynb-0.5.1-py3-none-any.whl (6.9 kB)


In [ ]:
pooling_analysis = pd.DataFrame()

events_fps = [#'s3://beam-outputs/pilates-outputs/sfbay_baseline_20230209/beam/year-2020-iteration-3/ITERS/it.0/0.events.csv.gz',
            #   'gs://beam-core-outputs/sfbay-2018-base-30pct-sample/beam/year-2019-iteration-1/ITERS/it.0/0.events.csv.gz',
            's3://beam-outputs/output/sfbay/oakland-rh-010__2023-02-12_10-34-24_ecx/ITERS/it.0/0.events.csv.gz',
            # 's3://beam-outputs/output/sfbay/oakland-rh-025__2023-02-12_17-37-28_pol/ITERS/it.0/0.events.csv.gz',
            # 's3://beam-outputs/output/sfbay/oakland-rh-050__2023-02-12_17-37-25_kgg/ITERS/it.0/0.events.csv.gz',
            # 's3://beam-outputs/output/sfbay/oakland-rh-100__2023-02-12_17-37-31_bup/ITERS/it.0/0.events.csv.gz',
            ]

names = ['Baseline']#, 'Baseline 30pct', 'Oakland 10pct', 'Oakland 25pct', 'Oakland 50pct', 'Oakland 100pct']

for event_fp, name in zip(events_fps, names):
    
    # PLANS
    plans = pd.read_csv(event_fp[:-13]+'plans.csv.gz', nrows = 50, usecols = ['planSelected','legMode'])
    # events=events.fillna(0)
    plans = plans[plans['planSelected']==True]
    plans_RH_solo =plans[plans['legMode']=='ride_hail']
    plans_RH_pooled =plans[plans['legMode']=='ride_hail_pooled']
    
    pooling_analysis.at['Planned RH Solo', name] = len(plans_RH_solo)
    pooling_analysis.at['Planned RH Pooled', name] = len(plans_RH_pooled)
    pooling_analysis.at['Planned RH Pooled share', name] = len(plans_RH_pooled)/(len(plans_RH_solo)+len(plans_RH_pooled))
    
    #EVENTS
    events = pd.read_csv(event_fp, nrows = 500000, usecols = ['type','vehicle','numPassengers','reason','mode','time'])
    
    #MC
    eventts_MC = events[events['type']=='ModeChoice'] 
    eventts_MC_solo = eventts_MC[eventts_MC['mode']=='ride_hail'] 
    eventts_MC_pooled = eventts_MC[eventts_MC['mode']=='ride_hail_pooled'] 
    eventts_MC_rhtr = eventts_MC[eventts_MC['mode']=='ride_hail_transit'] 
    
    pooling_analysis.at['Mode Choice RH Solo', name] = len(eventts_MC_solo)
    pooling_analysis.at['Mode Choice RH Pooled', name] = len(eventts_MC_pooled)
    pooling_analysis.at['Mode Choice RH tp transit', name] = len(eventts_MC_rhtr)
    pooling_analysis.at['Mode Choice RH Pooled share', name] = len(eventts_MC_pooled)/(len(eventts_MC_pooled)+len(eventts_MC_solo)+len(eventts_MC_rhtr))


    ## RP
    eventts_RP = events[events['type']=='Replanning'] 
    reasons = []
    for reason in eventts_RP['reason']:
        reasons.append(reason.split()[1].lower())
    eventts_RP['mode'] = reasons
    eventts_RP_solo = eventts_RP[eventts_RP['mode']=='ride_hail'] 
    eventts_RP_pooled = eventts_RP[eventts_RP['mode']=='ride_hail_pooled'] 
    
    pooling_analysis.at['Replanning RH Solo', name] = len(eventts_RP_solo)
    pooling_analysis.at['Replanning RH Pooled', name] = len(eventts_RP_pooled)
    pooling_analysis.at['Replanning RH Pooled share', name] = len(eventts_RP_pooled)/(len(eventts_RP_solo)+len(eventts_RP_pooled))

    pooling_analysis.at['Executed RH Solo', name] = pooling_analysis.at['Mode Choice RH Solo',name] - pooling_analysis.at['Replanning RH Solo',name]
    pooling_analysis.at['Executed RH Pooled', name] = pooling_analysis.at['Mode Choice RH Pooled',name] - pooling_analysis.at['Replanning RH Pooled',name]
    pooling_analysis.at['Executed RH Pooled share', name] = pooling_analysis.at['Executed RH Pooled',name]/(pooling_analysis.at['Executed RH Solo',name]+pooling_analysis.at['Executed RH Pooled',name])

    ## LEGS
    events_PT = events[events['type']=='PathTraversal']
    eventsRH = events_PT[events_PT['vehicle'].str.contains('ride')]
    agencies = []
    for vehicle in eventsRH['vehicle']:
        agencies.append(vehicle.split('@')[1])
    eventsRH['agency'] = agencies
    eventsRH['is_pool']=0
    eventsRH['is_pool'][eventsRH['numPassengers']>0]+=1
    eventsRH['is_pool'][eventsRH['numPassengers']>1]+=1
    eventsRH['is_pool'][eventsRH['numPassengers']>2]+=1
    eventsRH['is_pool'][eventsRH['numPassengers']>3]+=1
    eventsRH['is_pool'] = eventsRH['is_pool'].replace(0, 'empty')
    eventsRH['is_pool'] = eventsRH['is_pool'].replace(1, 'solo')
    eventsRH['is_pool'] = eventsRH['is_pool'].replace(2, 'pooled_2')
    eventsRH['is_pool'] = eventsRH['is_pool'].replace(3, 'pooled_3')
    eventsRH['is_pool'] = eventsRH['is_pool'].replace(4, 'pooled_4')
    
    pooling_analysis.at['Legs RH Empty', name] = len(eventsRH[eventsRH['is_pool']=='empty'])
    pooling_analysis.at['Legs RH Solo', name] = len(eventsRH[eventsRH['is_pool']=='solo'])
    pooling_analysis.at['Legs RH Pooled 2', name] = len(eventsRH[eventsRH['is_pool']=='pooled_2'])
    pooling_analysis.at['Legs RH Pooled 3', name] = len(eventsRH[eventsRH['is_pool']=='pooled_3'])
    pooling_analysis.at['Legs RH Pooled 4', name] = len(eventsRH[eventsRH['is_pool']=='pooled_4'])
    
    successfull_pooling = (pooling_analysis.at[ 'Legs RH Pooled 2',name]*2-pooling_analysis.at['Legs RH Pooled 3',name]+pooling_analysis.at[ 'Legs RH Pooled 4',name]*2)
    successfull_solo = pooling_analysis.at['Legs RH Solo',name] - pooling_analysis.at[ 'Legs RH Pooled 2',name]*2+pooling_analysis.at[ 'Legs RH Pooled 3',name]*2-pooling_analysis.at['Legs RH Pooled 4',name]*2
    successfull_rate_pool = successfull_pooling/pooling_analysis.at['Executed RH Pooled',name]
    successfull_rate_solo = successfull_solo/pooling_analysis.at['Executed RH Solo',name]
    
    pooling_analysis.at['Successfull Solo', name] = successfull_solo
    pooling_analysis.at['Successfull Pooling', name] = successfull_pooling
    pooling_analysis.at['Successfull Solo Rate', name] = successfull_rate_solo
    pooling_analysis.at['Successfull Pooling Rate', name] = successfull_rate_pool
    
    ## VEHICLES
    n_vehicle_trips = 0
    n_pooled_trips = 0
    n_solo_trips = 0
    for vehicle in list(np.unique(list(eventsRH['vehicle']))):
        eventsRH_vehicle = eventsRH[eventsRH['vehicle']==vehicle]
        print(vehicle)
        for stage in eventsRH_vehicle['numPassengers']:
            print(stage)
            


    
    #Includes per Agency data


In [ ]:
pooling_analysis.to_csv('outputs/pooling_analysis.csv')

In [4]:
#EVENTS
events = pd.read_csv('s3://beam-outputs/pilates-outputs/sfbay_baseline_20230209/beam/year-2020-iteration-3/ITERS/it.0/0.events.csv.gz', 
                     nrows = None, usecols = ['type','vehicle','numPassengers','reason','mode','time','person'])

## LEGS
events_PT = events[events['type']=='PathTraversal']
eventsRH = events_PT[events_PT['vehicle'].str.contains('ride')]

/tmp/ipykernel_88/80433486.py:2: DtypeWarning: Columns (6,9,33,35) have mixed types. Specify dtype option on import or set low_memory=False.
  events = pd.read_csv('s3://beam-outputs/pilates-outputs/sfbay_baseline_20230209/beam/year-2020-iteration-3/ITERS/it.0/0.events.csv.gz',


In [5]:
events

,vehicle,time,type,person,reason,mode,numPassengers
0,54,0.0,ParkingEvent,NaN,NaN,NaN,NaN
1,87,0.0,ParkingEvent,NaN,NaN,NaN,NaN
2,18,0.0,ParkingEvent,NaN,NaN,NaN,NaN
3,81,0.0,ParkingEvent,NaN,NaN,NaN,NaN
4,119,0.0,ParkingEvent,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...
36644238,NaN,214315.0,actstart,2763677.0,NaN,NaN,NaN
36644239,body-6769380,215595.0,PathTraversal,NaN,NaN,walk,1.0
36644240,body-6769380,215595.0,PersonLeavesVehicle,6769380.0,NaN,NaN,NaN
36644241,NaN,215595.0,arrival,6769380.0,NaN,NaN,NaN


In [6]:
dict_time = dict(zip(eventsRH.index,eventsRH.time))
dict_pass = dict(zip(eventsRH.index,eventsRH.numPassengers))

In [7]:
PtoPTs_RH = pd.read_csv('outputs/PtoPTss_RH_Baseline-20230209.csv', index_col=0)
eventts_MC = events[events['type']=='ModeChoice'] 
eventts_MC = eventts_MC.sort_values(by=['time'])

modes_available = eventts_MC.groupby('person').apply(lambda x: x.time).to_dict()


    

In [8]:
modes_available

{(4.0, 20372094): 54309.0,
 (4.0, 23266566): 58568.0,
 (4.0, 23484819): 58939.0,
 (8.0, 22776120): 57884.0,
 (8.0, 24336784): 60134.0,
 (18.0, 11511857): 39052.0,
 (18.0, 20393362): 54342.0,
 (66.0, 29437734): 66780.0,
 (66.0, 31074727): 68930.0,
 (66.0, 32291730): 71373.0,
 (113.0, 2106252): 23940.0,
 (113.0, 26090375): 62647.0,
 (114.0, 5347545): 29210.0,
 (114.0, 21220660): 55530.0,
 (115.0, 4891820): 28594.0,
 (115.0, 14700777): 45086.0,
 (115.0, 15907142): 47200.0,
 (116.0, 5579160): 29480.0,
 (116.0, 6096729): 30069.0,
 (116.0, 21014740): 55260.0,
 (116.0, 21972645): 56703.0,
 (116.0, 22661661): 57696.0,
 (116.0, 25068214): 61232.0,
 (116.0, 28479711): 65548.0,
 (125.0, 1764337): 22899.0,
 (125.0, 25770296): 62197.0,
 (126.0, 8808571): 33731.0,
 (126.0, 34320967): 76532.0,
 (145.0, 18985628): 52056.0,
 (145.0, 29608770): 67024.0,
 (149.0, 7822537): 32155.0,
 (149.0, 26895418): 63575.0,
 (149.0, 28573141): 65718.0,
 (152.0, 17397873): 49557.0,
 (152.0, 19047947): 52218.0,
 (153.0,

In [9]:
times = []
modes = []
num_pass = []
print(len(PtoPTs_RH))
i = 0
for Ptid, vehicle, person in zip(PtoPTs_RH['pathTraversalID'],PtoPTs_RH['vehicleID'],PtoPTs_RH['personID']):
    times.append(dict_time[Ptid])
    if i%100==0:
        print(i)
    i+=1
    num_pass.append(dict_pass[Ptid])
    modes_available = eventts_MC[(eventts_MC['person']==person)&(eventts_MC['time']<dict_time[Ptid])]
    modes.append(list(modes_available['mode'])[-1])
PtoPTs_RH['times'] = times
PtoPTs_RH['modes_MC'] = modes
PtoPTs_RH['num_pass'] = num_pass

PtoPTs_RH = PtoPTs_RH.sort_values(by=['times'])
PtoPTs_RH = PtoPTs_RH[PtoPTs_RH['modes_MC']=='ride_hail_pooled']

34493
0
100
200
300
400
500
600
700
800
900
1000
1100
1200
1300
1400
1500
1600
1700
1800
1900
2000
2100
2200
2300
2400
2500
2600
2700
2800
2900
3000
3100
3200
3300
3400
3500
3600
3700
3800
3900
4000
4100
4200
4300
4400
4500
4600
4700
4800
4900
5000
5100
5200
5300
5400
5500
5600
5700
5800
5900
6000
6100
6200
6300
6400
6500
6600
6700
6800
6900
7000
7100
7200
7300
7400
7500
7600
7700
7800
7900
8000
8100
8200
8300
8400
8500
8600
8700
8800
8900
9000
9100
9200
9300
9400
9500
9600
9700
9800
9900
10000
10100
10200
10300
10400
10500
10600
10700
10800
10900
11000
11100
11200
11300
11400
11500
11600
11700
11800
11900
12000
12100
12200
12300
12400
12500
12600
12700
12800
12900
13000
13100
13200
13300
13400
13500
13600
13700
13800
13900
14000
14100
14200
14300
14400
14500
14600
14700
14800
14900
15000
15100
15200
15300
15400
15500
15600
15700
15800
15900
16000
16100
16200
16300
16400
16500
16600
16700
16800
16900
17000
17100
17200
17300
17400
17500
17600
17700
17800
17900
18000
18100
18200
18300
18

In [13]:
PtoPTs_RH[PtoPTs_RH['vehicleID'] == 'rideHailVehicle-1023032@Uber']

,vehicleID,pathTraversalID,personID,planIndex,mode,length,duration,primaryFuelType,primaryFuel,secondaryFuel,vehicle2,times,modes_MC,num_pass
7626960,rideHailVehicle-1023032@Uber,930901,1014815,1,car_RideHail,2105.723,123,Gasoline,3.368649e+06,0.0,ri,18541.0,ride_hail_pooled,1.0
7626964,rideHailVehicle-1023032@Uber,3367118,5948265,1,car_RideHail,4322.145,270,Gasoline,6.887507e+06,0.0,ri,26526.0,ride_hail_pooled,1.0
7626972,rideHailVehicle-1023032@Uber,9248917,4659245,1,car_RideHail,15101.679,731,Gasoline,2.556777e+07,0.0,ri,34509.0,ride_hail_pooled,1.0
7626973,rideHailVehicle-1023032@Uber,10088584,5932906,1,car_RideHail,28701.467,1423,Gasoline,5.447096e+07,0.0,ri,36342.0,ride_hail_pooled,1.0


In [41]:
Succ_pool = 0
Not_succ_pool = 0
for vehicle in list(np.unique(PtoPTs_RH['vehicleID'])):
    PtoPTs_RH_veh = PtoPTs_RH[PtoPTs_RH['vehicleID'] == vehicle]
    on_boards = []
    on_board = []
    for path_id in list(np.unique(PtoPTs_RH_veh['pathTraversalID'])):
        PtoPTs_RH_veh_pathid = PtoPTs_RH_veh[PtoPTs_RH_veh['pathTraversalID']==path_id]
        for person in PtoPTs_RH_veh_pathid['personID']:
            on_board.append(person)
        on_boards.append(on_board)
        on_board = []
    pre_on_board = []
    trip = 0
    trips =[]
    for i in range(len(on_boards)):
        current_on_board = on_boards[i]
        is_new = 1
        for curr_on_bd in current_on_board:
            if curr_on_bd in pre_on_board:
                is_new = 0
        if current_on_board == pre_on_board:
            is_new = 1
        
        if is_new == 1:
            trip += 1
            trips.append(trip)
        else:
            trips.append(trip)
            
        pre_on_board = current_on_board

    tr_pre = -1
    is_pool = False
    for tr, ptid in zip(trips,list(np.unique(PtoPTs_RH_veh['pathTraversalID']))):
        num_pass = list(PtoPTs_RH_veh['num_pass'][PtoPTs_RH_veh['pathTraversalID']==ptid])
        

                
        if tr != tr_pre and tr_pre != -1:
            if is_pool:
                Succ_pool +=1
            else:
                Not_succ_pool +=1
            is_pool = False
            
        if 2 in num_pass or 3 in num_pass or 4 in num_pass:
             is_pool = True 
        tr_pre = tr
        
    if is_pool:
        Succ_pool +=1
    else:
        Not_succ_pool +=1
    is_pool = False

        
print('Pooling Success',Succ_pool)
print('Pooling no Success',Not_succ_pool)
print('Share of Pooling Success', Succ_pool/(Succ_pool+Not_succ_pool))

Pooling Success 581
Pooling no Success 3317
Share of Pooling Success 0.14905079527963058


In [35]:
Succ_pool = 0
Not_succ_pool = 0
for vehicle in list(np.unique(PtoPTs_RH['vehicleID'])):
    PtoPTs_RH_veh = PtoPTs_RH[PtoPTs_RH['vehicleID'] == vehicle]
    on_boards = []
    on_board = []
    for path_id in list(np.unique(PtoPTs_RH_veh['pathTraversalID'])):
        PtoPTs_RH_veh_pathid = PtoPTs_RH_veh[PtoPTs_RH_veh['pathTraversalID']==path_id]
        for person in PtoPTs_RH_veh_pathid['personID']:
            on_board.append(person)
        on_boards.append(on_board)
        on_board = []
    pre_on_board = []
    trip = 0
    trips =[]
    for i in range(len(on_boards)):
        current_on_board = on_boards[i]
        is_new = 1
        for curr_on_bd in current_on_board:
            if curr_on_bd in pre_on_board:
                is_new = 0
        if current_on_board == pre_on_board:
            is_new = 1
        
        if is_new == 1:
            trip += 1
            trips.append(trip)
        else:
            trips.append(trip)
            
        pre_on_board = current_on_board

    tr_pre = -1
    is_pool = False
    for tr in list(np.unique(trips)):
        trrr = trips.count(tr)
        if trrr >1:
            Succ_pool +=1
        else:
            Not_succ_pool+=1
        

                
        

        
print('Succ_pool',Succ_pool)
print('Not_succ_pool',Not_succ_pool)
print(Succ_pool/(Succ_pool+Not_succ_pool))

Succ_pool 581
Not_succ_pool 3317
0.14905079527963058


In [38]:
PtoPTs_RH[:59]

,vehicleID,pathTraversalID,personID,planIndex,mode,length,duration,primaryFuelType,primaryFuel,secondaryFuel,vehicle2,times,modes_MC,num_pass
7626960,rideHailVehicle-1023032@Uber,930901,1014815,1,car_RideHail,2105.723,123,Gasoline,3.368649e+06,0.0,ri,18541.0,ride_hail_pooled,1.0
7650358,rideHailVehicle-5094990@Uber,934890,4982594,1,car_RideHail,1830.438,99,Gasoline,2.904982e+06,0.0,ri,18559.0,ride_hail_pooled,1.0
7647094,rideHailVehicle-4562997@Lyft,934442,4745830,1,car_RideHail,2177.648,143,Gasoline,3.575903e+06,0.0,ri,18562.0,ride_hail_pooled,1.0
7640558,rideHailVehicle-3525407@Lyft,951643,2906264,3,car_RideHail,3604.164,222,Gasoline,5.873938e+06,0.0,ri,18689.0,ride_hail_pooled,1.0
7630149,rideHailVehicle-1708366@Uber,957415,2129694,1,car_RideHail,7282.275,323,Gasoline,1.365719e+07,0.0,ri,18760.0,ride_hail_pooled,1.0
7653556,rideHailVehicle-5583818@Uber,972111,5147019,1,car_RideHail,7715.158,379,Gasoline,1.391729e+07,0.0,ri,18851.0,ride_hail_pooled,1.0
7643467,rideHailVehicle-3974533@Uber,972509,4033720,3,car_RideHail,7823.993,354,Gasoline,1.545834e+07,0.0,ri,18877.0,ride_hail_pooled,1.0
7642964,rideHailVehicle-3902663@Uber,972995,3929795,1,car_RideHail,2337.182,124,Gasoline,4.583101e+06,0.0,ri,18892.0,ride_hail_pooled,1.0
7656708,rideHailVehicle-6167069@Uber,976091,6174272,1,car_RideHail,16224.748,588,Gasoline,2.979105e+07,0.0,ri,18901.0,ride_hail_pooled,1.0
7641392,rideHailVehicle-36443@Uber,991327,42725,3,car_RideHail,7033.839,305,Gasoline,1.315199e+07,0.0,ri,19026.0,ride_hail_pooled,1.0


In [55]:
PtoPTs_RH = PtoPTs_RH[PtoPTs_RH['modes_MC']=='ride_hail_pooled']
PtoPTs_RH_grouped = PtoPTs_RH.groupby(['vehicleID']).apply(lambda x: [list(x.personID),list(x.modes_MC),list(x.num_pass), list(x.times),list(x.pathTraversalID)]).to_dict()

In [54]:
PtoPTs_RH_grouped['rideHailVehicle-1007433@Uber',25159700]

[[923408], ['ride_hail_pooled'], [1.0], [61439.0], [25159700]]

In [56]:
PtoPTs_RH_grouped

{'rideHailVehicle-1007433@Uber': [[923408, 923408, 600180, 600180, 486348],
  ['ride_hail_pooled',
   'ride_hail_pooled',
   'ride_hail_pooled',
   'ride_hail_pooled',
   'ride_hail_pooled'],
  [1.0, 2.0, 2.0, 1.0, 1.0],
  [61439.0, 61754.0, 61754.0, 62845.0, 64111.0],
  [25159700, 25398056, 25398056, 26230499, 27284525]],
 'rideHailVehicle-1013349@Lyft': [[1599568, 1599568, 1620634],
  ['ride_hail_pooled', 'ride_hail_pooled', 'ride_hail_pooled'],
  [1.0, 1.0, 1.0],
  [22174.0, 23633.0, 26318.0],
  [1523319, 1993050, 3202741]],
 'rideHailVehicle-1017430@Uber': [[1215476],
  ['ride_hail_pooled'],
  [1.0],
  [32282.0],
  [7885214]],
 'rideHailVehicle-1023032@Uber': [[1014815, 5948265, 4659245, 5932906],
  ['ride_hail_pooled',
   'ride_hail_pooled',
   'ride_hail_pooled',
   'ride_hail_pooled'],
  [1.0, 1.0, 1.0, 1.0],
  [18541.0, 26526.0, 34509.0, 36342.0],
  [930901, 3367118, 9248917, 10088584]],
 'rideHailVehicle-1029101@Lyft': [[3214829, 3214829, 3131730, 3214829],
  ['ride_hail_poole

In [13]:
RH_stats = pd.DataFrame()

for vehicle in np.unique(PtoPTs_RH['vehicleID']):
    PtoPTs_RH_vehicle = PtoPTs_RH[PtoPTs_RH['vehicleID']==vehicle]
    agency = vehicle.split('@')[1]
    n_legs = len(np.unique(PtoPTs_RH_vehicle['pathTraversalID']))
    persons = PtoPTs_RH_grouped[vehicle][0]
    modes = PtoPTs_RH_grouped[vehicle][1]
    num_pass = PtoPTs_RH_grouped[vehicle][2]
    times = PtoPTs_RH_grouped[vehicle][3]
    pts_ids = PtoPTs_RH_grouped[vehicle][4]
    
    pathID_pre
    solo_requests = 0
    pooled_requests = 0pooled_success = 0
    
    current_on_board = []
 
    old_persons = []
    for pathID in pathIDS:
        new_persons = list(PtoPTs_RH['personID'][(PtoPTs_RH['pathTraversalID'] == pathID)])
        
        if 
        
        
    for pathID, mode, person in pts_ids:
        if mode is 'ride_hail':
            solo_requests += 1
            current_on_board = []
        elif  mode is 'ride_hail_pooled': 
            if current_on_board = []:
                current_on_board.append(person)
                pooled_requests +=1
            else:
                if pathID = pathID_pre:
                    current_on_board.append(person)
                else:
                    new_persons = list(PtoPTs_RH['personID'][(PtoPTs_RH['pathTraversalID'] == pathID)])
                    
                    is_new = 1
                    for new_person in new_persons:
                        if new_person in current_on_board:
                            is_new = 0
                    
                    if is_new = 1:
                        pooled_requests +=1
                        current_on_board = [person]
                    else:
                        current_on_board.append(person)

        pathID_pre = pathID
    

    
    
    
    
    
    
    

Uber 12
Uber 7
Lyft 10
Uber 10
Lyft 6
Uber 6
Uber 23
Lyft 30
Uber 21
Lyft 3
Uber 3
Lyft 12
Uber 11
Uber 8
Lyft 5
Uber 5
Lyft 23
Uber 30
Uber 7
Lyft 8
Uber 10
Lyft 12
Uber 14
Lyft 7
Uber 10
Lyft 14
Uber 18
Lyft 19
Uber 26
Uber 16
Lyft 5
Uber 6
Uber 15
Lyft 11
Uber 11
Uber 19
Uber 2
Uber 14
Uber 13
Uber 7
Lyft 9
Uber 9
Lyft 10
Uber 8
Uber 8
Lyft 17
Uber 16
Uber 8
Lyft 9
Uber 10
Uber 11
Uber 11
Uber 20
Lyft 25
Uber 38
Lyft 4
Uber 3
Uber 11
Lyft 5
Uber 7
Uber 19
Lyft 25
Uber 25
Lyft 14
Uber 20
Uber 32
Uber 8
Uber 11
Lyft 6
Uber 6
Lyft 9
Uber 14
Lyft 8
Uber 8
Lyft 8
Uber 9
Uber 18
Lyft 12
Uber 8
Lyft 20
Uber 19
Lyft 12
Uber 12
Lyft 18
Uber 14
Uber 9
Lyft 14
Uber 22
Lyft 13
Uber 19
Lyft 14
Uber 11
Lyft 10
Uber 14
Lyft 11
Uber 10
Lyft 8
Uber 7
Lyft 16
Uber 16
Lyft 12
Uber 12
Lyft 7
Uber 6
Lyft 6
Uber 16
Lyft 7
Uber 5
Uber 7
Lyft 15
Uber 12
Lyft 3
Uber 5
Uber 18
Uber 19
Lyft 10
Uber 9
Lyft 10
Uber 6
Uber 4
Lyft 16
Uber 13
Lyft 3
Uber 8
Uber 10
Lyft 2
Uber 6
Uber 18
Lyft 4
Uber 4
Lyft 17
Uber 1

KeyboardInterrupt: 

In [ ]:
PtoPTs_RH

In [ ]:
eventts_MC[eventts_MC['person']==5511463]
eventts_MC.groupby('vehicle').apply(lambda x: list(x.index)).to_dict()

In [ ]:
eventsRH[eventsRH.index == 24832464]